In [ ]:
!pip install -U openai -q
!pip install faiss-cpu -q # gpu 환경을 활용가능하면 faiss-gpu 로
!pip install sentence-transformers -q

In [ ]:
import pandas as pd
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import faiss
import openai, os
from google.colab import drive
drive.mount('/content/drive')

# CSV 파일 로드
df = pd.read_csv('/content/drive/MyDrive/Colab/사다리 담론 연구/df_filtered_final.csv', encoding='utf-8', lineterminator='\n')
df_sample = df.sample(n=500, random_state=42).reset_index(drop=True)

df_sample.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Date,Url,Title,Content,News,Style,Year,YearMonth,Char,Nouns
0,2020-10-08,https://www.hani.co.kr/arti/opinion/column/964...,[장석준의 그래도 진보정치] 그린 뉴딜에 빠진 ‘루카스 플랜’ 정신,장석준 ㅣ 전환사회연구소 기획위원 지난달 4일 영국의 공학자 마이크 쿨리가 향년 8...,Hankyoreh,Pro,2020,NaN,439.0,장석 전환 사회 연구소 기획 위원 지난달 영국 공학자 마이크 쿨리 향년 일기 세상 ...
1,2021.03.30,https://www.chosun.com/economy/economy_general...,2030 흙수저 공무원들 “등록할 재산 좀 있었으면...누구 놀리나”,"한국토지주택공사(LH) 직원들의 땅 투기 논란으로 정부가 모든 정부 부처, 공공기관...",Chosun,Con,2021,202103.0,368.0,한국토지주택공사 직원 투기 논란 정부 정부 부처 공공 기관 직원 재산 등록 의무 공...
2,2013-07-09,https://www.joongang.co.kr/article/12021077,"""대학 나와야지만 간호사 한다? 현대판 신분제도""",최근 논란이 되고 있는 ‘간호인력 경력상승제’에 대해 대한간호조무사협회(회장 강순심...,Joongang,Con,2013,NaN,210.0,논란 간호 인력 경력 상승 대한 간호조무사 협회 회장 강순 무협 찬성 입장 반대 간...
3,2013-09-18,https://www.joongang.co.kr/article/12639314,[사설] 번지수 잘못 짚은 중견기업 지원책,정부가 어제 중견기업을 육성하겠다며 ‘중견기업 성장사다리 구축 방안’을 내놓았다. ...,Joongang,Con,2013,NaN,225.0,정부 중견 기업 육성 중견 기업 성장 사다리 구축 방안 핵심 연구 개발 세액 공제 ...
4,2015-09-03,https://www.khan.co.kr/article/201509032102105,[로그인]현대판 음서제의 진실,높은 산이 병풍처럼 둘러싼 시골 초등학교·중학교에서 나는 선생님들로부터 은근한 후의...,Kyunghyang,Pro,2015,NaN,437.0,병풍 시골 초등학교 중학교 선생 은근 후의와 대접 성적 말썽 기억 꾸지람 당시 아버...


In [ ]:
# 텍스트만 추출
texts = df_sample["Content"].astype(str).tolist()

* dimension = embeddings.shape[1] : embeddings의 두 번째 축(열 개수=임베딩 벡터의 차원 숫자) 입력

* index = faiss.IndexFlatL2(dimension)
    - FAISS는 Facebook AI에서 만든 벡터 검색 라이브러리
    - IndexFlatL2: 가장 기본적인 인덱스 타입으로, L2 거리(유클리드 거리) 기반의 브루트포스 검색(빠른 근사 검색이 아니라, 정확한 거리 계산을 사용하는 단순 구조)

* dimension: 인덱스에 저장될 벡터의 차원 수를 지정합니다(위에서 구한 것과 같은 값)

* index.add(np.array(embeddings)) : 위에서 만든 벡터들을 벡터 데이터베이스(FAISS 인덱스)에 실제로 저장합니다.
    - 이후 index.search(query_vector, k) 식으로 가장 가까운 k개의 문장/문서 검색 가능

In [ ]:
# 임베딩 모델 로드
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")  # 사전학습된 문장 임베딩 모델 로드(한국어 전용, KoBERT 기반, 분류·STS 등 다목적)

# 임베딩 계산
embeddings = embedder.encode(texts, show_progress_bar=True)  # texts 리스트(문서/문장들)를 벡터로 변환, 진행 상황 바 표시

# FAISS 인덱스 구축
dimension = embeddings.shape[1]  # 임베딩 벡터의 차원 수 (예: 384)
index = faiss.IndexFlatL2(dimension)  # L2(유클리드 거리) 기반의 평면 인덱스 생성
index.add(np.array(embeddings))  # 계산된 모든 임베딩 벡터를 인덱스에 추가하여 벡터 DB 구축

print(f"벡터 데이터베이스 구축 완료 ({len(texts)}개 문서 저장됨)")  # 저장된 문서(벡터) 개수 출력

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

벡터 데이터베이스 구축 완료 (500개 문서 저장됨)


In [45]:
# 클라이언트 테스트
client = OpenAI(api_key="") # api 키 입력

# ChatCompletion 호출
response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "RAG가 무엇인지 간단히 설명해줘."}
    ]
)

# 결과 출력
print(response.choices[0].message.content)

RAG(Retrieval-Augmented Generation)는 대형언어모델(LLM)이 외부 지식을 실시간으로 검색해 그 결과를 바탕으로 응답을 생성하는 방법입니다. 간단히 정리하면:

- 핵심 아이디어: 모델이 자기 내부 기억만 쓰지 않고, 관련 문서(데이터베이스, 웹, 문서 저장소 등)를 먼저 검색(retrieval)하고, 그 검색 결과를 참고해서 응답을 생성(generation)함.
- 구성 요소: (1) 리트리버—질문과 관련된 문서를 찾음(벡터 검색·키워드 검색 등), (2) 제너레이터—검색한 문서를 입력으로 받아 답변 생성, (선택) 재순위기나 페이스티브(검증) 단계.
- 장점: 최신 정보나 도메인 지식 반영 가능, 근거 있는 응답(출처 제시)으로 사실성 향상.
- 한계: 검색 품질에 의존(잘못된 문서면 잘못된 답변), 응답 중 여전히 허위 생성(허영) 가능, 응답 지연·시스템 복잡성 증가.
- 활용 사례: 제품 Q&A, 기업 문서 기반 챗봇, 긴 문서 요약·질문 응답, 법률·의료 등 도메인 특화 응답.

원하면 간단한 동작 흐름(질문 → 검색 → 문서 선정 → 답변 생성)이나 구현 방법(벡터 검색, 파인튜닝, 캐시 등)도 설명해줄게요.


In [ ]:
# API 키는 코드에 직접 적지 않고, 환경변수에서 읽어오는 방식 권장
# - 터미널/Colab에서는 사전에:  os.environ["OPENAI_API_KEY"] = "발급받은_키"
# - 또는 .env, Secrets 기능 등을 사용해서 숨겨두고 관리
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # 환경변수에서 OPENAI_API_KEY를 읽어와 클라이언트 생성

# -------------------------------------------
# 검색 함수: FAISS 벡터 DB에서 문서 검색
# -------------------------------------------
def search_documents(query, k=3):
    """문서 검색: 사용자의 질의(query)와 가장 비슷한 문서 k개를 벡터 DB(FAISS)에서 찾아 반환"""
    query_vec = embedder.encode([query])          # 질의 문장을 임베딩 벡터로 변환 (리스트 형태여야 함)
    distances, indices = index.search(query_vec, k)  # FAISS 인덱스에서 질의 벡터와 가장 가까운 k개의 문서 검색
    return [texts[i] for i in indices[0]]         # 검색 결과의 인덱스를 이용해 원본 texts 리스트에서 문장 가져오기

# -------------------------------------------
# RAG 생성 함수: 검색 → 생성 (LLM 호출)
# -------------------------------------------
def rag_generate(query):
    """검색 → 생성: ① 질의와 관련된 문서 검색 → ② 그 문서를 바탕으로 LLM이 답변 생성"""
    docs = search_documents(query)          # 질의와 가장 유사한 문서들 검색
    context = "\n".join(docs)               # 여러 문서를 하나의 긴 문자열로 합쳐 컨텍스트로 사용

    # LLM에게 줄 프롬프트 구성: 컨텍스트 + 질문
    prompt = f"""
    아래는 '사다리 담론' 관련 기사 일부입니다.
    이 내용을 참고해 사용자의 질문에 대한 사회적 분석형 답변을 작성하세요.

    문서 내용:
    {context}

    질문:
    {query}
    """

    # 최신 OpenAI 클라이언트 문법으로 LLM 호출
    response = client.chat.completions.create(
        model="gpt-5-mini",  # 예시 모델 이름 (실제 사용 가능한 모델로 교체 필요: 예: "gpt-4o-mini")
        messages=[
            {
                "role": "system",
                "content": "당신은 사회 담론을 분석하는 AI 연구 어시스턴트입니다."
            },  # 시스템 메시지: AI의 역할/톤을 정의
            {
                "role": "user",
                "content": prompt
            }   # 유저 메시지: 컨텍스트와 질문이 합쳐진 최종 프롬프트
        ]
    )

    # 모델이 생성한 최종 답변 텍스트와, 함께 사용된 컨텍스트 문서들을 함께 반환
    return response.choices[0].message.content, docs

In [46]:
# -------------------------------------------
# 테스트 실행
# -------------------------------------------
query = "사다리 담론에서 청년 세대의 불평등 인식은 어떻게 나타나는가?"  # RAG에 던질 연구 질문(사용자 질의)

answer, docs = rag_generate(query)  # rag_generate 함수 호출: (1) 관련 문서 검색 → (2) LLM이 답변 생성

print("질문:", query)  # 질의 내용 출력
print("\nAI 요약 답변:\n", answer)  # RAG를 통해 생성된 AI의 요약/분석 답변 출력

print("\n참조 문서 샘플:")  # 어떤 문서들을 근거로 답변했는지 일부 보여주기
for d in docs:
    print("-", d[:250], "...")  # 각 문서의 앞부분 250자만 잘라서 요약 형태로 출력 (전체는 너무 길 수 있으므로)

질문: 사다리 담론에서 청년 세대의 불평등 인식은 어떻게 나타나는가?

AI 요약 답변:
 사다리 담론에서 청년 세대의 불평등 인식은 언어·경험·구조적 진단이 서로 맞물려 비교적 분명하게 드러난다. 기사 내용을 바탕으로 핵심 양상을 정리하면 다음과 같다.

- 일상 언어화된 불평등: ‘금수저·흙수저’ 같은 신조어가 대학생들이 꼽는 올해의 유행어로 등장할 정도로, 출발점(가정의 자본)이 삶의 경로를 결정한다는 인식이 널리 퍼져 있다. 이 표현은 기성세대의 ‘노이즈’가 아니라 젊은층의 예민한 현실인식을 반영한다.

- 교육과 취업에서의 칸막이 경험: 과거에는 ‘공부하면 출세한다’는 믿음이 상대적으로 유효했지만, 지금은 사교육·해외경험·비용 부담 등 가정자원이 교육기회·스펙으로 직결되며 로스쿨 사례처럼 교육·취업 경로가 사실상 세습화되는 것 아니냐는 의구심이 커졌다. “상위권 대학 출신도 취업 못하면 비빌 언덕(부모 회사 등)이 있는 사람과 없는 사람의 격차”라는 묘사가 그 감각을 보여준다.

- 자산·주거 불평등의 세대 효과: 1990년대 중후반 세계화 이후 자산 격차가 확대되면서 부동산·주식 등 자산 보유 여부가 청년들의 출발점을 결정한다는 인식이 확산되었다. 이 때문에 ‘사다리’가 내려앉았고 계층 이동 가능성이 줄어들었다는 불만이 강하다.

- 세대 간 경쟁과 노동시장 폐쇄성: 고령층의 노동 지속, 제조업 일자리 축소, 글로벌 경쟁 심화 등으로 청년들이 고임금·안정적 일자리를 얻기 어려워졌다. 이는 ‘독립=성년’이라는 규범이 붕괴되는 현실(캥거루족, 트윅스터 현상)과 맞물려, 청년들이 “예전보다 부모보다 못살기 쉬운 첫 세대”라는 인식을 낳는다.

- 심리·사회적 결과: 좌절감(삼포·N포세대), 위험 회피적 태도, 기성세대에 대한 분노와 불신, 정치적 요구(예: 기본소득·기본주택·기본금융 같은 정책 제안)의 확산으로 연결된다. 문화적 표현(오징어게임)과 정치인의 발언(청년이 ‘취약계층’이 되었다는 진단)도 이 인식을 공론장으로 끌어낸다.

- 역사적·제도적 성